# Pré-Processamento da Base de Dados - Instituições de Ensino Superior (IES):

## Importação das Bibliotecas:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pandas import factorize
import re

--------------------------------------------

## Obtenção dos Datasets:

In [7]:
df = pd.DataFrame()
df = pd.read_csv('../../data/trabalho2_dados10.csv')
df

,Codigo da IES,Natureza,Situacao,DIAS,%,CI N,CI V,IGC_N,IGC_M,IGC_V,EAD_17,variacao mat,mat_T 2017,% FIES,CURSOS,saldo 2017,Variacao 17-16
0,5124,1,Ativa,3941,0.411765,3,3,1.9541,2.0332,2.0332,1,1.207921,101,0.059406,2,2.404375e+08,77.190924
1,1313,0,Ativa,97,0.625000,4,3,2.3891,2.5598,2.5598,1,-0.048218,2385,0.017191,10,2.404375e+08,9.523631
2,13856,1,Ativa,2772,0.705882,3,3,2.102,2.102,2.102,0,0.000000,0,0.000000,0,0.000000e+00,0.000000
3,2380,1,Ativa,616,0.645161,3,3,2.7588,2.7945,2.7945,1,0.089311,2844,0.441280,6,8.838117e+08,0.060665
4,21685,1,Ativa,161,0.400000,S/C,S/C,S/D,S/D,S/D,0,0.000000,0,0.000000,-3,0.000000e+00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2172,21215,0,Ativa,516,1.000000,4,S/C,S/D,S/D,S/D,0,0.000000,0,0.000000,0,0.000000e+00,0.000000
2173,1465,1,Ativa,7042,0.370370,3,3,2.1593,2.4112,2.4112,1,0.524721,1254,0.614035,13,2.404375e+08,6.803505
2174,638,0,Ativa,885,0.600000,4,S/C,2.5174,2.6176,2.5861,1,-0.038257,1882,0.111052,1,9.297316e+06,0.662687
2175,13897,1,Ativa,2506,0.400000,3,3,2.3561,2.2901,0,1,-0.047619,357,0.000000,2,1.210978e+06,-0.000753


----------------------------------------

## Análises Pré-processamento:

### Descrição dos Dados:

    - COLUNA             DESCRIÇÃO                                    TYPE
    - ----------------------------------------------------------------------------------
    - Codigo da IES:   |  Id da instituição                          | Inteiro
    - Natureza:        |  Com ou sem fins lucrativos                 | Binário
    - Situação:        |  Ativa ou Extinta                           | Binário
    - DIAS:            |  Dias após o último ato institucional       | Inteiro
    - %:               |  % Cursos com mais de 5 anos sem AR         | Contínuo
    - CI N:            |  Conceito Institucional mais recente        | 1 a 5
    - CI V:            |  Penúltimo Conceito Institucional           | 1 a 5
    - IGC_N:           |  IGC mais recente                           | Inteiro
    - IGC_M:           |  Penúltimo IGC                              | Inteiro
    - IGC_V:           |  Antepenúltimo IGC                          | Inteiro
    - EAD_17:          |  Ofertou EAD em 2017                        | Binário
    - variacao_mat:    |  Variação de matrículas entre 2016 e 2017   | Contínuo
    - mat_T 2017:      |  Total de matrículas em 2017                | Inteiro
    - % FIES:          |  % de matrículas com FIES em 2017           | Contínuo
    - CURSOS:          |  Quantidade de cursos informados no censo   | Inteiro
    - saldo 2017:      |  Diferença entre receita e dispensa em 2017 | Contínuo
    - Variacao 17-16:  |  Variação do saldo entr 2016 e 2017         | Contínuo

---------------------------------------------

In [ ]:
### Quantidade de registros com valor ausente

In [3]:
# Verifique se qualquer coluna contém "S/D" em cada linha
contains_sd = df.apply(lambda row: row.str.contains('S/D').any(), axis=1)

# Calcule a porcentagem de linhas com pelo menos uma coluna contendo "S/D"
percentage_with_sd = (contains_sd.sum() / len(df)) * 100

print(f"A porcentagem de linhas com pelo menos uma coluna contendo 'S/D' é: {percentage_with_sd:.2f}%")

A porcentagem de linhas com pelo menos uma coluna contendo 'S/D' é: 31.24%


In [4]:
# Conte o número de ocorrências de 'S/D' em cada linha
count_sd = df.apply(lambda row: row.str.count('S/D').sum(), axis=1)

# Calcule a porcentagem de linhas com mais de uma ocorrência de 'S/D'
percentage_with_multiple_sd = (len(count_sd[count_sd > 1]) / len(df)) * 100

print(f"A porcentagem de linhas com mais de uma ocorrência de 'S/D' é: {percentage_with_multiple_sd:.2f}%")

A porcentagem de linhas com mais de uma ocorrência de 'S/D' é: 31.24%


In [ ]:
# Remova as colunas "Codigo da IES" e "DIAS"
df = df.drop(columns=["Codigo da IES", "DIAS"])

# Selecione todas as colunas numéricas restantes
numeric_columns = df.select_dtypes(include=['number'])

# Verifique se todos os valores nas colunas numéricas são iguais a zero
all_zeros = (numeric_columns == 0).all(axis=1)

# Calcule a porcentagem de linhas onde todas as colunas numéricas (exceto "Codigo da IES" e "DIAS") são iguais a zero
percentage_all_zeros = (all_zeros.sum() / len(df)) * 100

print(f"A porcentagem de linhas onde todas as colunas numéricas (exceto 'Codigo da IES' e 'DIAS') são iguais a zero é: {percentage_all_zeros:.2f}%")

In [ ]:
# Crie um DataFrame temporário excluindo as colunas "Codigo da IES", "DIAS", "Natureza" e "Situacao"
df_temp = df.drop(columns=["Codigo da IES", "DIAS", "Natureza", "Situacao"])

# Verifique se "S/D" ocorre em cada linha (retorna True/False)
contains_sd = df_temp.apply(lambda row: row.str.contains('S/D').any(), axis=1)

# Calcule a porcentagem de colunas em que "S/D" ocorre para cada linha
percentage_sd_per_row = (df_temp[contains_sd].apply(lambda row: (row == 'S/D').sum(), axis=1) / df_temp[contains_sd].shape[1]) * 100

# Calcule a média das porcentagens
average_percentage_sd = percentage_sd_per_row.mean()

print(f"A média da porcentagem de colunas em que 'S/D' ocorre nas linhas (ignorando 'Codigo da IES', 'DIAS', 'Natureza' e 'Situacao') é: {average_percentage_sd:.2f}%")

------------------------------------------

### Verificando Inconsistências:

In [9]:
df.dtypes

Codigo da IES       int64
Natureza            int64
Situacao           object
DIAS                int64
%                 float64
CI N               object
CI V               object
IGC_N              object
IGC_M              object
IGC_V              object
EAD_17              int64
variacao mat      float64
mat_T 2017          int64
% FIES            float64
CURSOS              int64
saldo 2017        float64
Variacao 17-16    float64
dtype: object

In [10]:
df_1 = df[['Codigo da IES', 'Natureza', 'DIAS', '%', 'EAD_17', 'variacao mat', 'mat_T 2017', '% FIES', 
           'CURSOS', 'saldo 2017', 'Variacao 17-16']]

df_2 = df[['Situacao', 'CI N', 'CI V', 'IGC_N', 'IGC_M', 'IGC_V']]

In [13]:
df_1.isna().sum()

Codigo da IES     0
Natureza          0
DIAS              0
%                 0
EAD_17            0
variacao mat      0
mat_T 2017        0
% FIES            0
CURSOS            0
saldo 2017        0
Variacao 17-16    0
dtype: int64

In [23]:
df_1.isin(['S/C']).sum(axis=0)

Codigo da IES     0
Natureza          0
DIAS              0
%                 0
EAD_17            0
variacao mat      0
mat_T 2017        0
% FIES            0
CURSOS            0
saldo 2017        0
Variacao 17-16    0
dtype: int64

In [24]:
df_1.isin(['S/D']).sum(axis=0)

Codigo da IES     0
Natureza          0
DIAS              0
%                 0
EAD_17            0
variacao mat      0
mat_T 2017        0
% FIES            0
CURSOS            0
saldo 2017        0
Variacao 17-16    0
dtype: int64

In [14]:
df_2.isna().sum()

Situacao      0
CI N          0
CI V          0
IGC_N         0
IGC_M         0
IGC_V       185
dtype: int64

In [20]:
df_2.isin(['S/C']).sum(axis=0)

Situacao       0
CI N         371
CI V        1089
IGC_N          0
IGC_M          0
IGC_V          0
dtype: int64

In [21]:
df_2.isin(['S/D']).sum(axis=0)

Situacao      0
CI N         98
CI V         98
IGC_N       679
IGC_M       678
IGC_V       618
dtype: int64

-------------------------------------